# 快速开始

介绍如何使用 nwpc-data 库。

首先导入需要使用的一些库。

In [3]:
import numpy as np
import pandas as pd
import eccodes

## 获取业务系统产品文件路径

可以使用内置的配置文件获取 CMA-PI 或二级存储中业务系统产品文件路径。

下面的示例获取 GRAPES GFS GMF 系统 2020 年 4 月 22 日 00 时次 024 时效的原始分辨率 GRIB 2 文件路径。

In [10]:
from nwpc_data.data_finder import find_local_file

file_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h",
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042121/ORIG/gmf.gra.2020042200024.grb2')

使用 `number` 参数指定集合预报的成员编号。

下面示例获取 GRAPES GEPS 系统第 20 个成员 2020 年 4 月 22 日 12 时次 048 时效的原始分辨率 GRIB 2 文件路径。

In [11]:
file_path = find_local_file(
    "grapes_geps/grib2/orig",
    start_time=pd.to_datetime("2020-04-22 12:00:00"),
    forecast_time=pd.Timedelta(hours=48),
    number=20,
)
file_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GEPS/Prod-grib/2020042212/grib2/gef.gra.020.2020042212048.grb2')

nwpc-data 还内置了共享数据集的部分数据。

获取 FNL 资料。

In [29]:
file_path = find_local_file(
    "glob/fnl",
    start_time="2019122000",
    data_class="cm",
)
file_path

PosixPath('/sstorage1/COMMONDATA/glob/fnl/2019/fnl_20191220_00_00.grib2')

内置的配置文件请查看 `nwpc_data/data_finder/conf` 目录

## 检索 GRIB 2 消息

nwpc-data 使用 [eccodes-python](https://github.com/ecmwf/eccodes-python) 从 GRIB 2 文件中检索需要的要素场，并返回 GRIB 2 消息。

下面的示例从文件中检索 850hPa 温度场。

In [12]:
data_path = find_local_file(
    "grapes_gfs_gmf/grib2/orig",
    start_time="2020042200",
    forecast_time="24h",
)
data_path

PosixPath('/sstorage1/COMMONDATA/OPER/NWPC/GRAPES_GFS_GMF/Prod-grib/2020042121/ORIG/gmf.gra.2020042200024.grb2')

In [13]:
from nwpc_data.grib.eccodes import load_message_from_file

t850 = load_message_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

94734264112368

可以使用 eccodes-python 的 API 接口获取 GRIB 2 消息的属性和数据。

In [14]:
print("shortName:   ", eccodes.codes_get(t850, "shortName"))
print("typeOfLevel: ", eccodes.codes_get(t850, "typeOfLevel"))
print("level:       ", eccodes.codes_get(t850, "level"))

shortName:    t
typeOfLevel:  isobaricInhPa
level:        850


In [15]:
values = eccodes.codes_get_double_array(t850, "values")
values = values.reshape([720, 1440])
values

array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])

**注意**：需要手动调用 `eccodes.codes_release` 释放消息对象。

In [16]:
eccodes.codes_release(t850)

## 加载 GRIB 2 要素场

nwpc-data 还提供对上述检索得到 GRIB 2 消息的封装，返回 `xarray.DataArray` 对象，类似 [cfgrib](https://github.com/ecmwf/cfgrib) 库。

In [17]:
from nwpc_data.grib.eccodes import load_field_from_file

t850 = load_field_from_file(
    data_path,
    parameter="t",
    level_type="pl",
    level=850,
)
t850

<xarray.DataArray 't' (latitude: 720, longitude: 1440)>
array([[258.75289063, 258.76289063, 258.76289063, ..., 258.75289063,
        258.76289063, 258.75289063],
       [258.47289062, 258.48289062, 258.46289062, ..., 258.46289062,
        258.47289062, 258.44289063],
       [258.27289063, 258.26289063, 258.27289063, ..., 258.25289063,
        258.26289063, 258.26289063],
       ...,
       [227.68289062, 227.68289062, 227.72289062, ..., 227.62289062,
        227.75289062, 227.63289063],
       [227.90289062, 227.90289062, 227.90289062, ..., 227.90289062,
        227.88289063, 227.91289063],
       [228.12289062, 228.18289062, 228.19289063, ..., 228.09289062,
        228.23289063, 228.20289063]])
Coordinates:
    time        datetime64[ns] 2020-04-22
    step        timedelta64[ns] 1 days
    valid_time  datetime64[ns] 2020-04-23
    pl          float64 850.0
  * latitude    (latitude) float64 89.88 89.62 89.38 ... -89.38 -89.62 -89.88
  * longitude   (longitude) float64 0.0 0.25 0.5 0.75 ... 359.2 359.5 359.8
Attributes:
    GRIB_edition:                    2
    GRIB_centre:                     babj
    GRIB_subCentre:                  0
    GRIB_tablesVersion:              4
    GRIB_localTablesVersion:         1
    GRIB_dataType:                   fc
    GRIB_dataDate:                   20200422
    GRIB_dataTime:                   0
    GRIB_validityDate:               20200423
    GRIB_validityTime:               0
    GRIB_step:                       24
    GRIB_stepType:                   instant
    GRIB_stepUnits:                  1
    GRIB_stepRange:                  24
    GRIB_endStep:                    24
    GRIB_name:                       Temperature
    GRIB_shortName:                  t
    GRIB_cfName:                     air_temperature
    GRIB_discipline:                 0
    GRIB_parameterCategory:          0
    GRIB_parameterNumber:            0
    GRIB_gridType:                   regular_ll
    GRIB_gridDefinitionDescription:  Latitude/longitude 
    GRIB_typeOfFirstFixedSurface:    pl
    GRIB_typeOfLevel:                isobaricInhPa
    GRIB_level:                      850
    GRIB_numberOfPoints:             1036800
    GRIB_missingValue:               9999
    GRIB_units:                      K
    long_name:                       Temperature
    units:                           K

`t850` 对象已对 GRIB 2 消息进行解码，包含两个维度：纬度（`latitude`）和经度（`longitude`）。

可以使用 xarray 库提供的一系列工具对数据进行处理。比如，求纬向平均值。

In [20]:
t850.mean(dim="longitude")

<xarray.DataArray 't' (latitude: 720)>
array([258.89689063, 258.92158507, 258.9702934 , 259.03200868,
       259.13369618, 259.25227951, 259.36137674, 259.43795313,
       259.48117535, 259.51569618, 259.54527951, 259.58036285,
       259.61993229, 259.66970313, 259.73472396, 259.8143559 ,
       259.90958507, 260.01493924, 260.12211979, 260.23601562,
       260.33834201, 260.43259896, 260.52713368, 260.62855035,
       260.73437674, 260.8412934 , 260.95026562, 261.07211979,
       261.20684201, 261.33564757, 261.47109201, 261.61936979,
       261.77783507, 261.94189062, 262.09409896, 262.24932812,
       262.38886979, 262.47721007, 262.54587674, 262.62440451,
       262.69916146, 262.75284201, 262.78774479, 262.80559896,
       262.81419618, 262.82962674, 262.84596007, 262.86258507,
       262.88671701, 262.89856424, 262.91805729, 262.93743229,
       262.96751563, 262.98624479, 262.98996701, 262.9832309 ,
       262.97455729, 262.96696701, 262.95727257, 262.9509184 ,
       262.94531424, 262.92838368, 262.92327257, 262.90801562,
       262.89346701, 262.89584896, 262.89827257, 262.92536979,
       262.95557118, 262.98227951, 263.00800174, 263.05213368,
       263.1318559 , 263.24283507, 263.35055729, 263.44265451,
       263.53114757, 263.61167535, 263.69661285, 263.76896007,
       263.83617535, 263.87674479, 263.98159896, 264.10572396,
       264.26920312, 264.44573785, 264.61956424, 264.7910434 ,
       264.9632934 , 265.10965451, 265.22516146, 265.34021007,
       265.47066146, 265.61875174, 265.76950868, 265.91295313,
       266.04075868, 266.15169618, 266.25124479, 266.36041146,
       266.52465451, 266.69387674, 266.86807118, 267.03982813,
       267.21142535, 267.40073785, 267.5943559 , 267.75667535,
       267.89971007, 268.05203646, 268.21666146, 268.38387674,
       268.54184201, 268.7090434 , 268.87580729, 268.99101562,
       269.11693229, 269.23324479, 269.33961979, 269.46050174,
       269.58415451, 269.72396701, 269.86667535, 270.01165451,
       270.14057118, 270.25455729, 270.3506059 , 270.44326562,
       270.52814757, 270.61777257, 270.70231424, 270.79052257,
       270.88641146, 271.00218229, 271.13405035, 271.28502257,
       271.4329809 , 271.58130729, 271.72655729, 271.84647396,
       271.97452257, 272.10697396, 272.22942535, 272.33621701,
       272.43039757, 272.51641146, 272.58534896, 272.66176562,
       272.7420434 , 272.81633507, 272.86363368, 272.90052951,
       272.94570313, 273.00134201, 273.02714063, 273.04571007,
       273.08643924, 273.17557813, 273.27590451, 273.38655035,
       273.48040451, 273.55252951, 273.60469618, 273.6549809 ,
       273.71563368, 273.79621007, 273.88759896, 273.97005035,
       274.06513368, 274.14778646, 274.20848785, 274.26907813,
       274.3411059 , 274.46211979, 274.57980035, 274.69694618,
       274.81127951, 274.93432812, 275.06308507, 275.19967535,
       275.35625174, 275.51227951, 275.65175174, 275.82320313,
       275.95711285, 276.07559896, 276.21678646, 276.37627257,
       276.53001563, 276.6933559 , 276.90605729, 277.09180729,
       277.28546007, 277.48648785, 277.66282118, 277.85827257,
       278.04939757, 278.21332812, 278.41915451, 278.64528646,
       278.89750868, 279.09636285, 279.29001563, 279.50972396,
       279.70877257, 279.89778646, 280.1213559 , 280.3346684 ,
       280.5513559 , 280.74786285, 280.94209896, 281.11664063,
       281.27112674, 281.42489757, 281.58027257, 281.72990451,
       281.88550868, 282.06462674, 282.25342535, 282.42500174,
       282.61556424, 282.8092309 , 283.00943229, 283.1751684 ,
       283.35144618, 283.52786979, 283.69568229, 283.86865451,
       284.0544809 , 284.25872396, 284.49625174, 284.69396701,
       284.87019618, 285.04240451, 285.21368229, 285.39555729,
       285.56309896, 285.7232309 , 285.9014809 , 286.10871007,
       286.28134201, 286.4457934 , 286.60746007, 286.78932812,
       286.96641146, 287.15187674, 287.36663368, 287.58855035,
       287.77125174, 287.91775174, 288.08464063, 288.29

## 下一步

请浏览下一章节 [介绍和安装](./01_introduction.ipynb)。